# The Battle of Neighborhoods - Notebook 
## Introduction

Venturing into a business in hospitality/food industry can be tricky. The list of things to keep in mind before owning a business in the sector is quite long. One among them is choose the locality in such a manner that it helps in the business's long term profitability and it builds a strong foothold in the area of choice. While being a resident of a city may help with a surface level knowledge of the hip localites initially, there is lots that might get missed when relying on personal experiences alone. This is where data comes in. 

We can leverage information found on the Foursquare API to segment the neighbourhoods of Delhi based on the restaurants in the area and accordingly give recommendations backed by data to a new business owner looking to open a restaurant in the city. The information can also be helpful to customers for finding popular neighbourhoods for different kinds of food joint. 

### Business Problem: A new business owner is looking to open a restaurant in Delhi. Where would you recommend they open it?

## Data

All data related to restaurants and venues in Delhi are procured by Foursquare API which is a location application that recommends nearby venues and local businesses in the user's area. All data is retrieved in JSON format and then converted to a dataframe in Pandas.
The information on the neighbourhoods of Delhi are scraped from the Wikipedia page of the same name. There are total 139 neighbourhoods in Delhi. 
To call a query from the Foursquare API, we need the latitude and longitude coordinates of the neighbourhoods. To get the coordinates data, we use the library geopy.geocoders. 

## Methodology 

In [119]:
#Import Libraries

import requests #Library to make url requests 
from bs4 import BeautifulSoup #Library for web scraping 
import numpy as np #Library to handle data in vectorized manner
import pandas as pd # Library for data analysis
pd.set_option('display.max_columns', None) #To list all columns
pd.set_option('display.max_rows', None) #To list all rows
import json #Library to handle JSON files 
from pandas.io.json import json_normalize #tranform JSON file into a pandas dataframe
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans # import k-means from clustering stage
import folium #Library to visualise geo spatial data

### Scraping neighbourhood data for Delhi from Wikipedia page 

In [4]:
url = 'https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Delhi'
response = requests.get(url)

In [5]:
soup = BeautifulSoup(response.text, 'html.parser')

In [6]:
li = soup.findAll('div', class_="mw-category-group")

In [7]:
Neighbourhood = []
for i in range(1,len(li)):
    cells = li[i]
    
    for cell in cells:
        link = cells.findAll('a')
        
        for n in range(0, len(link)):
            neigh = link[n]
            Neighbourhood.append(neigh['title'])
        
        

In [8]:
delhi = pd.DataFrame(Neighbourhood, columns = ['Neighbourhood'])

In [9]:
delhi.drop_duplicates(inplace = True)

In [10]:
delhi.reset_index(inplace = True)

In [121]:
de = delhi.drop(['index'], axis = 1)

In [122]:
print(de.shape) 
de.head()

(139, 1)


,Neighbourhood
0,Ashok Nagar (Delhi)
1,Ashok Vihar
2,Ashram Chowk
3,Babarpur
4,"Badarpur, Delhi"


### Getting latitude and longitude coordinates 

In [13]:
geolocator = Nominatim(user_agent="de_explorer")
def lat_long_coord(neigh):
    location = geolocator.geocode('{}, Delhi, India'.format(neigh), timeout = 10000)
    try:
        latitude = location.latitude
        longitude = location.longitude
    except: 
        latitude = 'none'
        longitude = 'none'
    return latitude, longitude

In [14]:
lat_long = de['Neighbourhood'].apply(lat_long_coord)

In [15]:
de['Latitude'], de['Longitude'] = zip(*lat_long)

In [17]:
de = de[de.Latitude != 'none']

In [18]:
de.reset_index(drop = True, inplace = True)

In [126]:
print(de.shape)
de.head(10)

(127, 3)


,Neighbourhood,Latitude,Longitude
0,Ashok Nagar (Delhi),28.636,77.1018
1,Ashok Vihar,28.6995,77.1848
2,Ashram Chowk,28.6831,77.1901
3,Babarpur,28.6874,77.2798
4,"Badarpur, Delhi",28.4932,77.303
5,Bali Nagar,28.6541,77.1282
6,Bawana,28.7997,77.0329
7,Ber Sarai,28.5484,77.1815
8,Bhajanpura,28.701,77.261
9,Chanakyapuri,28.5947,77.1885


### Exploring venues around each neighbourhood using Foursquare API 

In [43]:
#Defining Foursquare credentials and version, and specifying the number of venues to show within the radius 
CLIENT_ID = 'AHIKRYX4LHKACT11VDRGSV1ECXVJD3IJ4YH4H0UF3IQIKD5E' # your Foursquare ID
CLIENT_SECRET = 'CQWRVKUU5CQYHXTEPQRAXJ1UAZKDBGHZZD332ACKDWBZ2YAQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 500

In [51]:
venues = []

for lat, long, neighborhood in zip(de['Latitude'], de['Longitude'], de['Neighbourhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [127]:
de_venues = pd.DataFrame(venues)

# define the column names
de_venues.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

de_venues.head(10)

,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Ashok Nagar (Delhi),28.636021,77.101822,McDonald's,28.636260,77.097104,Fast Food Restaurant
1,Ashok Nagar (Delhi),28.636021,77.101822,Giani Ice Cream,28.638913,77.101122,Ice Cream Shop
2,Ashok Nagar (Delhi),28.636021,77.101822,Haldiram's tilak nagar,28.638989,77.101133,North Indian Restaurant
3,Ashok Nagar (Delhi),28.636021,77.101822,CCD tilak nagar,28.636264,77.097048,Café
4,Ashok Nagar (Delhi),28.636021,77.101822,Grillz Jail Road,28.635471,77.097031,Fast Food Restaurant
5,Ashok Vihar,28.699453,77.184826,ELIXIR FERTILITY CENTRE,28.700034,77.188093,Health & Beauty Service
6,Ashok Vihar,28.699453,77.184826,Bakers Stop,28.700495,77.188716,Bakery
7,Ashok Vihar,28.699453,77.184826,Invitation Banquet,28.696018,77.185953,Diner
8,Ashok Vihar,28.699453,77.184826,Gola Northend,28.701242,77.189288,Indian Restaurant
9,Ashram Chowk,28.683134,77.190083,S Square,28.683455,77.187857,Chinese Restaurant


In [160]:
#Checking the count of categories in each Neighborhood
count_df = de_venues.groupby('Neighborhood').count() 
count_df.head(10)

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Ashok Nagar (Delhi),5,5,5,5,5,5
Ashok Vihar,4,4,4,4,4,4
Ashram Chowk,2,2,2,2,2,2
Babarpur,3,3,3,3,3,3
"Badarpur, Delhi",3,3,3,3,3,3
Bali Nagar,4,4,4,4,4,4
Bawana,1,1,1,1,1,1
Ber Sarai,7,7,7,7,7,7
Bhajanpura,1,1,1,1,1,1


##### We notice that the number of categories returned in each neighborhood is quite skewed. Since foursquare API returns local businesses and attractions, we may assume that neighbourhoods with categories less than 4 are areas with little commercial avenues. Hence, any owner of a potential restaurant would not be interested in these areas. Therefore, we continue our analysis of Delhi by removing these areas from our consideration.  

In [139]:
#List of neighborhoods with venue categories less than 4 
delete_row = count_df[count_df.VenueCategory < 4]
li = list(delete_row.index) 

In [149]:
#Removing the list of neighborhoods from our venue dataframe
de_venue = de_venues.loc[~de_venues.Neighborhood.isin(li)]
de_venue.head(10)

,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Ashok Nagar (Delhi),28.636021,77.101822,McDonald's,28.636260,77.097104,Fast Food Restaurant
1,Ashok Nagar (Delhi),28.636021,77.101822,Giani Ice Cream,28.638913,77.101122,Ice Cream Shop
2,Ashok Nagar (Delhi),28.636021,77.101822,Haldiram's tilak nagar,28.638989,77.101133,North Indian Restaurant
3,Ashok Nagar (Delhi),28.636021,77.101822,CCD tilak nagar,28.636264,77.097048,Café
4,Ashok Nagar (Delhi),28.636021,77.101822,Grillz Jail Road,28.635471,77.097031,Fast Food Restaurant
5,Ashok Vihar,28.699453,77.184826,ELIXIR FERTILITY CENTRE,28.700034,77.188093,Health & Beauty Service
6,Ashok Vihar,28.699453,77.184826,Bakers Stop,28.700495,77.188716,Bakery
7,Ashok Vihar,28.699453,77.184826,Invitation Banquet,28.696018,77.185953,Diner
8,Ashok Vihar,28.699453,77.184826,Gola Northend,28.701242,77.189288,Indian Restaurant
17,Bali Nagar,28.654138,77.128178,Udyog Bhawan Metro Station,28.654029,77.129623,Light Rail Station


In [145]:
# one hot encoding
onehot_de = pd.get_dummies(de_venue[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot_de['Neighborhood'] = de_venue['Neighborhood'] 

onehot_de.head()

,ATM,Accessories Store,Airport Food Court,American Restaurant,Antique Shop,Arcade,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bistro,Boutique,Breakfast Spot,Burger Joint,Burmese Restaurant,Bus Station,Bus Stop,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Electronics Store,English Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Historic Site,History Museum,Hookah Bar,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Men's Store,Metro Station,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mosque,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue,Neighborhood,Nightclub,Nightlife Spot,North Indian Restaurant,Other Nightlife,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Playground,Plaza,Portuguese Restaurant,Pub,Racetrack,Restaurant,Road,Sake Bar,Salon / Barbershop,Sandwich Place,Sculpture Garden,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,South Indian Restaurant,Spa,Spiritual Center,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Tibetan Restaurant,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Ashok Nagar (Delhi),0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [146]:
#Show 'Neighborhood' column in 1st place 
onehot_de = onehot_de[ ['Neighborhood'] + [ col for col in onehot_de.columns if col != 'Neighborhood' ] ]

In [193]:
onehot_de.head(10)

,Neighborhood,ATM,Accessories Store,Airport Food Court,American Restaurant,Antique Shop,Arcade,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bistro,Boutique,Breakfast Spot,Burger Joint,Burmese Restaurant,Bus Station,Bus Stop,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Electronics Store,English Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Historic Site,History Museum,Hookah Bar,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Men's Store,Metro Station,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mosque,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue,Nightclub,Nightlife Spot,North Indian Restaurant,Other Nightlife,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Playground,Plaza,Portuguese Restaurant,Pub,Racetrack,Restaurant,Road,Sake Bar,Salon / Barbershop,Sandwich Place,Sculpture Garden,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,South Indian Restaurant,Spa,Spiritual Center,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Tibetan Restaurant,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Women's Store
0,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Ashok Vihar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,Ashok Vihar,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Ashok Vihar,0,0,0,0,0,0,0,0

In [195]:
#grouping the categories in each neighborhood by mean of frequency of occurence of each category  
de_grouped = onehot_de.groupby('Neighborhood').mean().reset_index()
de_grouped

,Neighborhood,ATM,Accessories Store,Airport Food Court,American Restaurant,Antique Shop,Arcade,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bistro,Boutique,Breakfast Spot,Burger Joint,Burmese Restaurant,Bus Station,Bus Stop,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Electronics Store,English Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Historic Site,History Museum,Hookah Bar,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Men's Store,Metro Station,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mosque,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue,Nightclub,Nightlife Spot,North Indian Restaurant,Other Nightlife,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Playground,Plaza,Portuguese Restaurant,Pub,Racetrack,Restaurant,Road,Sake Bar,Salon / Barbershop,Sandwich Place,Sculpture Garden,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,South Indian Restaurant,Spa,Spiritual Center,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Tibetan Restaurant,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Women's Store
0,Ashok Nagar (Delhi),0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.400000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Ashok Vihar,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000

In [81]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [151]:
num_top_venues = 4

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = de_grouped['Neighborhood']

for ind in np.arange(de_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(de_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
0,Ashok Nagar (Delhi),Fast Food Restaurant,Ice Cream Shop,North Indian Restaurant,Café
1,Ashok Vihar,Indian Restaurant,Health & Beauty Service,Bakery,Diner
2,Bali Nagar,Indian Restaurant,Light Rail Station,Chinese Restaurant,Ice Cream Shop
3,Ber Sarai,Shopping Mall,Hotel,Airport Food Court,History Museum
4,Chandni Chowk,Indian Restaurant,Snack Place,Market,Jewelry Store
5,Chittaranjan Park,Market,Pizza Place,Fast Food Restaurant,Women's Store
6,"Civil Lines, Delhi",Indian Restaurant,Smoke Shop,Italian Restaurant,Light Rail Station
7,"Connaught Place, New Delhi",Indian Restaurant,Café,Chinese Restaurant,Bar
8,"Dabri, New Delhi",Movie Theater,Department Store,Grocery Store,Food & Drink Shop
9,Dariba Kalan,Indian Restaurant,Market,Snack Place,Hotel


### Clustering the neighborhoods 

In [152]:
# set number of clusters
kclusters = 3

de_grouped_clustering = de_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(de_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 2, 1, 2, 1, 1, 2, 1, 2])

In [154]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# add clustering labels
de_merged = de

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
de_merged = de_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

de_merged.dropna(axis = 0, inplace = True) #Removing neighborhoods that were deleted previously
de_merged.head() 

,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
0,Ashok Nagar (Delhi),28.636,77.1018,1.0,Fast Food Restaurant,Ice Cream Shop,North Indian Restaurant,Café
1,Ashok Vihar,28.6995,77.1848,2.0,Indian Restaurant,Health & Beauty Service,Bakery,Diner
5,Bali Nagar,28.6541,77.1282,2.0,Indian Restaurant,Light Rail Station,Chinese Restaurant,Ice Cream Shop
7,Ber Sarai,28.5484,77.1815,1.0,Shopping Mall,Hotel,Airport Food Court,History Museum
10,Chandni Chowk,28.656,77.2322,2.0,Indian Restaurant,Snack Place,Market,Jewelry Store


In [155]:
# create map
map_clusters = folium.Map(location=[lat, long], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(de_merged['Latitude'], de_merged['Longitude'], de_merged['Neighbourhood'], de_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [196]:
de_merged[de_merged['Cluster Labels'] == 0]

,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
27,Geetanjali Enclave,28.5304,77.2021,0.0,Café,Metro Station,Coffee Shop,Women's Store
35,Gulmohar Park,28.5571,77.213,0.0,Park,Theater,Café,Music Venue
37,Janakpuri,28.6219,77.0875,0.0,Park,Gym / Fitness Center,Café,Bakery
62,Mayur Vihar Phase - 3,28.6115,77.3363,0.0,ATM,Music Venue,Martial Arts Dojo,Burger Joint
63,Meera Bagh,28.6588,77.0908,0.0,ATM,Convenience Store,Spa,Café
70,Najafgarh,28.6123,76.9824,0.0,ATM,Food & Drink Shop,Mobile Phone Shop,Hookah Bar
71,Nanakpura,28.5839,77.1714,0.0,Department Store,Asian Restaurant,Café,Bus Stop
75,Naveen Shahdara,28.6773,77.286,0.0,ATM,Men's Store,Gym / Fitness Center,Vegetarian / Vegan Restaurant
80,Nizamuddin East,28.5897,77.2504,0.0,Train Station,Park,Café,Food Court
89,Patel Nagar,28.6598,77.157,0.0,Women's Store,Café,ATM,Train Station


In [197]:
de_merged[de_merged['Cluster Labels'] == 1]


,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
0,Ashok Nagar (Delhi),28.636,77.1018,1.0,Fast Food Restaurant,Ice Cream Shop,North Indian Restaurant,Café
7,Ber Sarai,28.5484,77.1815,1.0,Shopping Mall,Hotel,Airport Food Court,History Museum
11,Chittaranjan Park,28.5388,77.2492,1.0,Market,Pizza Place,Fast Food Restaurant,Women's Store
12,"Civil Lines, Delhi",28.6769,77.225,1.0,Indian Restaurant,Smoke Shop,Italian Restaurant,Light Rail Station
14,"Dabri, New Delhi",28.6108,77.0911,1.0,Movie Theater,Department Store,Grocery Store,Food & Drink Shop
18,Dayanand Colony,28.5622,77.2476,1.0,Restaurant,Indie Movie Theater,Fast Food Restaurant,Market
21,Derawal Nagar,28.6998,77.1901,1.0,Shopping Mall,Bakery,Snack Place,Pizza Place
22,Dhaula Kuan,28.5919,77.1617,1.0,Burger Joint,Sake Bar,Golf Course,Bakery
25,East Patel Nagar,28.6456,77.1726,1.0,Pizza Place,Hotel,Donut Shop,Bed & Breakfast
28,Ghitorni,28.4938,77.1492,1.0,Indian Restaurant,Pizza Place,Sandwich Place,Sporting Goods Shop


In [198]:
de_merged[de_merged['Cluster Labels'] == 2]

,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
1,Ashok Vihar,28.6995,77.1848,2.0,Indian Restaurant,Health & Beauty Service,Bakery,Diner
5,Bali Nagar,28.6541,77.1282,2.0,Indian Restaurant,Light Rail Station,Chinese Restaurant,Ice Cream Shop
10,Chandni Chowk,28.656,77.2322,2.0,Indian Restaurant,Snack Place,Market,Jewelry Store
13,"Connaught Place, New Delhi",28.6314,77.2198,2.0,Indian Restaurant,Café,Chinese Restaurant,Bar
15,Dariba Kalan,28.6546,77.2334,2.0,Indian Restaurant,Market,Snack Place,Hotel
16,Daryaganj,28.6461,77.243,2.0,Indian Restaurant,Restaurant,Hotel,Road
19,Defence Colony,28.5712,77.2318,2.0,Indian Restaurant,Italian Restaurant,Bakery,Bar
20,Delhi Cantonment,28.5938,77.135,2.0,Indian Restaurant,Coffee Shop,Shopping Mall,Café
23,Dilshad Garden,28.6758,77.3215,2.0,Indian Restaurant,Light Rail Station,Diner,Pizza Place
24,"Dwarka, Delhi",28.5772,77.0443,2.0,Indian Restaurant,Accessories Store,IT Services,Grocery Store


### Results 

Among 93 neighborhoods of Delhi, we see a total of 14 neighborhoods in the 1st cluster (numbered 0), 46 neighborhoods in 2nd cluster (numbered 1) and 33 neighborhoods in 3rd cluster (numbered 2).

### Discussion

In the 1st cluster, among the popular venues are ATM, department/convenience stores, cafes and other utility businesses or parks. We can gauge that the cluster mostly consists of neighbourhoods with local market places having small or fast food joints among food venues.  

The 2nd cluster, with the most number of venues offers very diverse categories. It consists of a vast number of food businesses like restaurants with different range of cuisines, cafes, pizzerias, ice cream shops, bakery etc. Coupled with food venues, are other hospitality businesses or tourist attractions (Eg. Hotels, Multiplex, Museums etc.)

The 3rd cluster contains neighborhoods heavily popular for Indian restaurants. It also consists of a number of other restaurants and food joints, however the cuisines offered are very limited apart from Indian. Among the top 2 most common venues, food businesses are most common. It also has a number of lodging businesses like hotels and bed and breakfasts, but not much to offer in terms of other tourist attractions. 

**Recommendations to business owner :**
Out of the 3 clusters, the 2nd cluster (numbered 1) seems the most popular and conducive for any bussinessperson looking to get into restaurant business. The area is promising because the preferences are diverse in terms of customer's selection of the kind of food joint they wish to eat at (Restaurants, Cafes, Bakery etc.) and the cuisine preference is also diverse and so the owner may not be restricted in their business choices. The areas have strong tourist footfall owing to a number of attractions and lodging avenues offered, so there seems to be promising profitability for any new business to come up. 

The 3rd cluster (numbered 2), although popular for restaurants has heavy inclination towards Indian cuisine. Any businessperson looking to open an Indian restaurant may consider the areas, however, there will be high competition and poaching customers from already established Indian restaurants may be tough. Businessperson coming with a franchise that has repute in similar localities in a different city may consider to open a restaurant here. 

The 1st cluster (numbered 0) does not seem to be a cluster with a lot of businesses in the hospitality sector that may attract customers. Their food preference are limited to fast food joints, that suggests that most customers don't visit the areas for a dine-in experience (like Restaurants, Cafes etc.). Other factors considered, if a businessperson is looking to take the risk and get the first mover advantage may invest in a restaurant in this cluster. 

 ### Conclusion

The safest best for a businessperson looking to open a restaurant in the city are the neighbourhoods of 2nd cluster (numbered 1). 